In [25]:
import numpy as np
import pandas as pd
import matplotlib.pyplot as plt
import re
import os
import sys
import urllib, json, requests

In [2]:
protest_data = pd.read_csv("../raw_data/protest_AECLD.csv")

In [3]:
protest_data.head()

,ISO,EVENT_ID_CNTY,EVENT_ID_NO_CNTY,EVENT_DATE,YEAR,TIME_PRECISION,EVENT_TYPE,SUB_EVENT_TYPE,ACTOR1,ASSOC_ACTOR_1,...,ADMIN2,ADMIN3,LOCATION,LATITUDE,LONGITUDE,GEO_PRECISION,SOURCE,SOURCE_SCALE,NOTES,FATALITIES
0,840,USA4524,4524,24-May-2020,2020,1,Protests,Peaceful protest,Protesters (United States),NaN,...,Baltimore City,NaN,Baltimore,39.290,-76.612,1,Fox45 (Baltimore),Subnational,"On 24 May 2020, a caravan of people staged a m...",0
1,840,USA4525,4525,24-May-2020,2020,1,Protests,Peaceful protest,Protesters (United States),NaN,...,Williamson,NaN,Marion,37.731,-88.933,1,Southern Illinoisan,Subnational,"On 24 May 2020, nearly 50 people held a ""Patri...",0
2,840,USA4526,4526,24-May-2020,2020,1,Protests,Peaceful protest,Protesters (United States),NaN,...,Mahoning,NaN,Youngstown,41.100,-80.650,1,WKBN,Subnational,"On 24 May 2020, an unreported number of people...",0
3,840,USA4527,4527,24-May-2020,2020,1,Protests,Peaceful protest,Protesters (United States),Teachers (United States); Latinx Group (United...,...,Middlesex,NaN,New Brunswick,40.486,-74.452,1,My Central Jersey,Subnational,"On 24 May 2020, more than a dozen vehicles, or...",0
4,840,USA4528,4528,24-May-2020,2020,1,Protests,Peaceful protest,Protesters (United States),NaN,...,Essex,NaN,Keene,44.256,-73.792,1,Post-Star,Subnational,"On 24 May 2020, a few people staged a protest ...",0


In [171]:
sr = r'\[size=(.*)\]'
nums = r'\d+'
sizes = protest_data.NOTES.str.findall(sr)
size_temp_df = pd.DataFrame([''.join(x) for x in sizes], columns = ['string'])
size_nums = size_temp_df.string.str.findall(nums)

size_cleaned = []
for i in size_nums:
    size_cleaned.append(np.mean(np.array(i, dtype=np.float32).astype(np.float)))

protest_data["size"] = size_cleaned

/Users/Sapthau/opt/anaconda3/envs/cs109a/lib/python3.7/site-packages/numpy/core/fromnumeric.py:3373: RuntimeWarning: Mean of empty slice.
  out=out, **kwargs)
/Users/Sapthau/opt/anaconda3/envs/cs109a/lib/python3.7/site-packages/numpy/core/_methods.py:170: RuntimeWarning: invalid value encountered in double_scalars
  ret = ret.dtype.type(ret / rcount)


In [27]:
def progressbar(n_step, n_total):
    """Prints self-updating progress bar to stdout to track for-loop progress
    
    There are entire 3rd-party libraries dedicated to custom progress-bars.
    A simple function like this is often more than enough to get the job done.
    
    :param n_total: total number of expected for-loop iterations
    :type n_total: int
    :param n_step: current iteration number, starting at 0
    :type n_step: int

    .. example::
    
        for i in range(n_iterations):
            progressbar(i, n_iterations)
            
    .. source:
    
        This function is a simplified version of code found here:
        https://stackoverflow.com/questions/3160699/python-progress-bar/15860757#15860757
    """
    n_step = n_step + 1
    barlen = 50
    progress = n_step / n_total
    block = int(round(barlen * progress))
    status = ""
    if n_step == n_total:
        status = "Done...\r\n\n"
    text = "\r [{0}] {1}/{2} {3}".format(
        "=" * block + "-" * (barlen - block),
        n_step,
        n_total,
        status,
    )
    sys.stdout.write(text)
    sys.stdout.flush()

In [46]:
fips_codes = []

for i in range(len(protest_data)):
    
    temp_url = "https://geo.fcc.gov/api/census/area?lat=" +str(protest_data.LATITUDE[i]) +"&lon=" + str(protest_data.LONGITUDE[i])+ "&format=json"
    
    with urllib.request.urlopen(temp_url) as url:
        data = json.loads(url.read().decode())
    
    if not data["results"]:
        fips_codes.append("")
    else:
        fips_codes.append(data["results"][0]["county_fips"])
        
    progressbar(i, len(protest_data))

print(fips_codes)

 [==================================================] 16880/16880 Done...

['24510', '17199', '39099', '34023', '36031', '36081', '36031', '27053', '17031', '37119', '21073', '37133', '42039', '34029', '25025', '33015', '04013', '17163', '27053', '12073', '26161', '18089', '26161', '06059', '48201', '06037', '42101', '51760', '18141', '27123', '27137', '06107', '17031', '24027', '26161', '27053', '41051', '27163', '26161', '37119', '12021', '45003', '27137', '24015', '06037', '47157', '36061', '37183', '06073', '06067', '42027', '53007', '29095', '40143', '36119', '08031', '27163', '26161', '26161', '27053', '25017', '08069', '08031', '35001', '39049', '27123', '26161', '41051', '19113', '01089', '41039', '26161', '06071', '27003', '27123', '27123', '06037', '27053', '29510', '53061', '27037', '47157', '27037', '23011', '48245', '01073', '27003', '25025', '12081', '27035', '17031', '45079', '27137', '48141', '41039', '41067', '42001', '06001', '26065', '32003', '32003', '06037', '21111

In [47]:
protest_data["fips_county"] = fips_codes

In [54]:
protest_data.EVENT_DATE = pd.to_datetime(protest_data.EVENT_DATE)
protest_data = protest_data.replace('', np.nan)

In [173]:
cleaned_protest_data = protest_data[["EVENT_DATE", "fips_county", "size"]]
cleaned_protest_data.to_csv("../processed_data/protests.csv", index = False)